In [106]:
import cv2 as cv 
import matplotlib.pyplot as plt 
import numpy as np 
import math

In [107]:
image_path = 'download (1).jpeg'

# Load the image
image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred = cv.GaussianBlur(image, (5, 5), 0)

# Canny edge detection
edges = cv.Canny(blurred, 50, 180)

# Display the original image and the edges
cv.imshow('Original Image', image)
cv.imshow('Canny Edges', edges)
cv.waitKey(0)
cv.destroyAllWindows()

# OpenCV Circle

In [108]:

# Load the image
image = cv.imread('download (1).jpeg', cv.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred_image = cv.GaussianBlur(image, (5, 5), 0)




# Helper Functions

In [109]:
from collections import defaultdict

class IntersectionPoints:
    def __init__(self):
        self.points = defaultdict(int)

    def add_point(self, point):
        self.points[point] += 1

    def get_intersection_points(self):
        return [(point, count) for point, count in self.points.items()]

def circle_intersection(x1, y1, r1, x2, y2, r2):
    # Calculate the distance between the centers of the circles
    d = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # Check if the circles are separate or coincident
    if d > r1 + r2 or d < abs(r1 - r2):
        return None  # No intersection

    # Calculate the intersection points
    a = (r1**2 - r2**2 + d**2) / (2 * d)
    h = math.sqrt(r1**2 - a**2)
    x3 = x1 + a * (x2 - x1) / d
    y3 = y1 + a * (y2 - y1) / d
    x4 = x3 + h * (y2 - y1) / d
    y4 = y3 - h * (x2 - x1) / d

    # If the circles are coincident, return just one intersection point
    if d == abs(r1 + r2):
        return [(x3, y3)]

    return [(x3, y3), (x4, y4)]

# Example usage
intersection_points = IntersectionPoints()
x1, y1, r1 = 0, 0, 4
x2, y2, r2 = 4, 0, 4
points = circle_intersection(x1, y1, r1, x2, y2, r2)
if points:
    for point in points:
        intersection_points.add_point(point)

print("Intersection points and counts:", intersection_points.get_intersection_points())


Intersection points and counts: [((2.0, 0.0), 1), ((2.0, -3.4641016151377544), 1)]


# From Scratch Implementation

In [110]:
import numpy as np

#dp to control accumulator size, min_dist to control accepted circles according to the distance bet. their centers
def hough_circle_transform(image, min_radius, max_radius, accumulator_threshold, dp, min_dist):
    # Define Hough space dimensions based on image size and radius range
    height, width = image.shape
    # Define resolution of the accumulator
    accumulator_height = int(height / dp)
    accumulator_width = int(width / dp)
    # Define number of radii
    num_radii = max_radius - min_radius + 1
    # Initialize the 3D Hough space to specify (a, b) for each radius later on
    accumulator = np.zeros((accumulator_height, accumulator_width, num_radii), dtype=np.uint64)
    
    # Define radius range for iteration
    radii = np.arange(min_radius, max_radius + 1)
    
    # Find edge pixels in the image to get (x,y) in image space
    edge_pixels = np.argwhere(image > 0)
    for x, y in edge_pixels:
        # Iterate over a range of radii
        for radius in range(min_radius, max_radius + 1):
            # Calculate center (a, b) using the parametric form of the circle equation
            for theta in np.linspace(0, 2*np.pi, 100):  # Iterate over angles for a full circle
                a = int(x - radius * np.cos(theta))
                b = int(y - radius * np.sin(theta))
                
                # Boundary check for accumulator array
                if 0 <= b < accumulator.shape[0] and 0 <= a < accumulator.shape[1]:
                    # Increment accumulator at the position corresponding to the center (a, b)
                    accumulator[b, a, radius - min_radius] += 1

    # Identify circle candidates by selecting peaks in the accumulator array
    circle_candidates = np.argwhere(accumulator >= accumulator_threshold)

    # Filter circle candidates based on minimum distance between centers
    filtered_circle_candidates = []
    for y, x, radius_idx in circle_candidates:
        radius = min_radius + radius_idx
        valid_circle = True
        for y2, x2, radius2_idx in filtered_circle_candidates:
            radius2 = min_radius + radius2_idx
            distance = np.sqrt((x - x2)**2 + (y - y2)**2)
            if distance < min_dist:
                valid_circle = False
                break
        if valid_circle:
            filtered_circle_candidates.append((y, x, radius))

    return filtered_circle_candidates



min_radius = 10
max_radius = 50
accumulator_threshold = 1000
dp = 1
min_dist = 100  # Adjust this value as needed

circle_candidates = hough_circle_transform(edges, min_radius, max_radius, accumulator_threshold, dp, min_dist)

# Draw detected circles on the grayscale version of the original image
for y, x, radius in circle_candidates:
    cv.circle(image, (x, y), radius, (0, 0, 0), 3)


# Detect circles using Hough Circle Transform
circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, dp=1, minDist=20,
                           param1=50, param2=30, minRadius=10, maxRadius=50)

# If circles are detected, draw them
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        cv.circle(image, (x, y), r, (0, 0, 0), 2)

# Display the image with detected circles
# Display the image with detected circles
cv.imshow("Our Detected Circles", image)
cv.imshow("Detected Circles (OpenCV)", image)
cv.waitKey(0)
cv.destroyAllWindows()
